In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as st 
from tqdm import tqdm 
import os
## Change directory to your working directory
os.chdir('D:\Shashi Katteri\Desktop\data sets to work on\Amazon Employee Access\Dataset_10')

In [2]:
df=pd.read_csv('Amazon_employee_access.csv')
df.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,target
0,39353,85475,117961,118300,123472,117905,117906,290919,117908,1
1,17183,1540,117961,118343,123125,118536,118536,308574,118539,1
2,36724,14457,118219,118220,117884,117879,267952,19721,117880,1
3,36135,5396,117961,118343,119993,118321,240983,290919,118322,1
4,42680,5905,117929,117930,119569,119323,123932,19793,119325,1


In [3]:
df1=df.astype(str)
df1['target']=df1['target'].astype(int)

In [4]:
#df1['ROLE_ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_TITLE','ROLE_FAMILY'],axis=1)


In [5]:
df1['target']=df1['target'].map({1:0,0:1})

In [6]:
df1['ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_ROLLUP_1','ROLE_ROLLUP_2'],axis=1)

In [7]:
df1.head()

,RESOURCE,MGR_ID,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target,ROLLUP_12
0,39353,85475,123472,117906,117908,0,117961118300
1,17183,1540,123125,118536,118539,0,117961118343
2,36724,14457,117884,267952,117880,0,118219118220
3,36135,5396,119993,240983,118322,0,117961118343
4,42680,5905,119569,123932,119325,0,117929117930


# Combinations with brute force 


In [8]:
from itertools import combinations

cols=list(df1.columns)
cols.remove('target')
d=df1.copy()

for j in combinations(cols,2):
    d[j[0]+'_'+j[1]]=df1[j[0]]+df1[j[1]]
    
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32769 entries, 0 to 32768
Data columns (total 22 columns):
RESOURCE                          32769 non-null object
MGR_ID                            32769 non-null object
ROLE_DEPTNAME                     32769 non-null object
ROLE_FAMILY_DESC                  32769 non-null object
ROLE_CODE                         32769 non-null object
target                            32769 non-null int64
ROLLUP_12                         32769 non-null object
RESOURCE_MGR_ID                   32769 non-null object
RESOURCE_ROLE_DEPTNAME            32769 non-null object
RESOURCE_ROLE_FAMILY_DESC         32769 non-null object
RESOURCE_ROLE_CODE                32769 non-null object
RESOURCE_ROLLUP_12                32769 non-null object
MGR_ID_ROLE_DEPTNAME              32769 non-null object
MGR_ID_ROLE_FAMILY_DESC           32769 non-null object
MGR_ID_ROLE_CODE                  32769 non-null object
MGR_ID_ROLLUP_12                  32769 non-null obj

In [9]:
def AST_ecoding(train,test,columns,target,weight):
    t=train.copy()
    v=test.copy()
    gm=t[target].mean()
    for col in columns:
        gb=t.groupby(col)[target].agg(['count','mean'])
        count=gb['count']
        mean=gb['mean']
        gb['smooth']=(count*mean + weight*gm)/(count+weight)
        t[col+'_enc']=t[col].map(gb['smooth'])
        
        v[col+'_enc']=v[col].map(gb['smooth'])
        v[col+'_enc']=v[col+'_enc'].fillna(gm)
    v=v.drop(columns,axis=1)
    t=t.drop(columns,axis=1)
    return t,v

In [10]:
def aucroc(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=accuracy_score(y,pred)
    return auc
    #print('AUC_ROC= ',auc)
    #print('Accuracy= ',acc)
    #print('confusion matrix:\n',confusion_matrix(y,pred))

In [11]:
from lightgbm import LGBMClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
lgb=LGBMClassifier(n_estimators=200)
lr=LogisticRegression()

In [12]:
train,test=train_test_split(d,test_size=0.3,random_state=123)

columns=list(d.columns)
columns.remove('target')
target='target'
t,v=AST_ecoding(train,test,columns,target,20)

In [13]:
t.shape

(22938, 22)

In [14]:
t=t.sample(n=10000)
t[target].value_counts()

0    9415
1     585
Name: target, dtype: int64

In [15]:

xtr=t.drop('target',axis=1)
xts=v.drop(target,axis=1)
ytr=t[target]
yts=v[target]
cols=list(xtr.columns)


In [29]:
def eliminate(algo,cols,xtr,ytr,xts,yts):
    mod=algo.fit(xtr[cols],ytr)
    auc=aucroc(mod,xts[cols],yts)
    
    high=0

    for i in tqdm(cols):
        c=cols.copy()
        c.remove(i)

        mod=algo.fit(xtr[c],ytr)
        auc_sc=aucroc(mod,xts[c],yts)
        if auc_sc>high:
            high=auc_sc
            r=i

    return high,r


In [17]:
algo=lr
removed=[]
cols=list(xtr.columns)
mod=algo.fit(xtr[cols],ytr)
auc=aucroc(mod,xts[cols],yts)
high_current=auc
high_new,r=eliminate(algo,cols,xtr,ytr,xts,yts)
while high_new>high_current:
    cols.remove(r)
    removed.append(r)
    high_current=high_new
    high_new,r=eliminate(algo,cols,xtr,ytr,xts,yts)
    print(high_new)
print(high_current)
print(removed)

100%|██████████| 20/20 [00:01<00:00, 18.65it/s]


0.8765686170675522
0.8769879110196579
['ROLE_CODE_ROLLUP_12_enc']


In [35]:
def col_select(algo,step):
    
    
    cols=list(xtr.columns)
    p=[]
    removed=[]
    mod=algo.fit(xtr[cols],ytr)
    auc=aucroc(mod,xts[cols],yts)
    high_current=auc
    removed_new=[]
    high_new=auc
    best=auc
   # best_rem=[]
    while high_current-high_new<step:    


        removed=removed+removed_new
        print(len(removed))
        removed_new=[]


        if high_current==auc:
            pass
        else:
            high_current=high_new
        if len(cols)>2:
            high_new,r=eliminate(algo,cols,xtr,ytr,xts,yts)
            while high_new>=high_current:
                cols.remove(r)
                removed_new.append(r)
                high_current=high_new

                high_new,r=eliminate(algo,cols,xtr,ytr,xts,yts)
                print(high_new)
                if high_new>best:
                    best=high_new
                    best_rem=removed+removed_new+[r]
        else:
            break

    print('high_auc=',high_current)
    print('auc_one step ahead=',high_new)
    print('removed: ',removed)
    print('len=',len(removed))
    print('best=',best)
    print('best_rem=',best_rem)

In [36]:
col_select(lr,0.05)

0


100%|██████████| 20/20 [00:01<00:00, 17.46it/s]


0.8779787459385036


100%|██████████| 19/19 [00:01<00:00, 18.89it/s]


0.8787310920555428


...

0.8756016121162146
17


100%|██████████| 3/3 [00:00<00:00, 24.60it/s]


0.8518487137491348
18


100%|██████████| 2/2 [00:00<00:00, 23.84it/s]


0.8104457527811085
19
high_auc= 0.8104457527811085
auc_one step ahead= 0.8104457527811085
removed:  ['RESOURCE_ROLE_CODE_enc', 'ROLE_FAMILY_DESC_ROLLUP_12_enc', 'MGR_ID_enc', 'ROLE_DEPTNAME_ROLE_FAMILY_DESC_enc', 'MGR_ID_ROLLUP_12_enc', 'RESOURCE_ROLE_FAMILY_DESC_enc', 'ROLE_DEPTNAME_ROLE_CODE_enc', 'MGR_ID_ROLE_DEPTNAME_enc', 'ROLE_FAMILY_DESC_enc', 'ROLE_CODE_ROLLUP_12_enc', 'ROLE_DEPTNAME_enc', 'ROLE_DEPTNAME_ROLLUP_12_enc', 'ROLLUP_12_enc', 'RESOURCE_ROLE_DEPTNAME_enc', 'MGR_ID_ROLE_CODE_enc', 'ROLE_CODE_enc', 'RESOURCE_MGR_ID_enc', 'RESOURCE_ROLLUP_12_enc', 'ROLE_FAMILY_DESC_ROLE_CODE_enc']
len= 19
best= 0.8848271192595311
best_rem= ['RESOURCE_ROLE_CODE_enc', 'ROLE_FAMILY_DESC_ROLLUP_12_enc', 'MGR_ID_enc', 'ROLE_DEPTNAME_ROLE_FAMILY_DESC_enc', 'MGR_ID_ROLLUP_12_enc', 'RESOURCE_ROLE_FAMILY_DESC_enc', 'ROLE_DEPTNAME_ROLE_CODE_enc', 'MGR_ID_ROLE_DEPTNAME_enc', 'ROLE_FAMILY_DESC_enc', 'ROLE_CODE_ROLLUP_12_enc', 'ROLE_DEPTNAME_enc', 'ROLE_DEPTNAME_ROLLUP_12_enc', 'ROLLUP_12_enc']


In [38]:
def aucroc1(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=accuracy_score(y,pred)
    #return auc
    print('AUC_ROC= ',auc)
    print('Accuracy= ',acc)
    print('confusion matrix:\n',confusion_matrix(y,pred))

In [39]:
rem=['RESOURCE_ROLE_CODE_enc', 'ROLE_FAMILY_DESC_ROLLUP_12_enc', 'MGR_ID_enc', 'ROLE_DEPTNAME_ROLE_FAMILY_DESC_enc',
     'MGR_ID_ROLLUP_12_enc', 'RESOURCE_ROLE_FAMILY_DESC_enc', 'ROLE_DEPTNAME_ROLE_CODE_enc', 'MGR_ID_ROLE_DEPTNAME_enc',
     'ROLE_FAMILY_DESC_enc', 'ROLE_CODE_ROLLUP_12_enc', 'ROLE_DEPTNAME_enc', 'ROLE_DEPTNAME_ROLLUP_12_enc', 'ROLLUP_12_enc']
lr=LogisticRegression()
t1=t.drop(rem,axis=1)
v1=v.drop(rem,axis=1)
xtr=t1.drop('target',axis=1)
xts=v1.drop(target,axis=1)
ytr=t1[target]
yts=v1[target]
#cols=list(xtr.columns)
mod=lr.fit(xtr,ytr)
aucroc1(mod,xts,yts)

AUC_ROC=  0.8848271192595311
Accuracy=  0.9456820262435154
confusion matrix:
 [[9215   45]
 [ 489   82]]


In [88]:
xtr.head()

,RESOURCE_enc,ROLE_CODE_enc,RESOURCE_MGR_ID_enc,RESOURCE_ROLE_DEPTNAME_enc,RESOURCE_ROLLUP_12_enc,MGR_ID_ROLE_FAMILY_DESC_enc,MGR_ID_ROLE_CODE_enc,ROLE_FAMILY_DESC_ROLE_CODE_enc
8690,0.014452,0.006801,0.055055,0.033033,0.021814,0.044468,0.044468,0.026887
2267,0.081493,0.024739,0.055055,0.034005,0.052553,0.048173,0.048173,0.063123
13666,0.053904,0.055286,0.055055,0.089840,0.038539,0.052553,0.052553,0.052553
25586,0.078310,0.048173,0.055055,0.053494,0.046246,0.050268,0.050268,0.050268
11882,0.052553,0.035277,0.055055,0.055055,0.052553,0.048173,0.042821,0.046246


In [87]:
len(rem)

13

In [60]:
prob=mod.predict_proba(xts)[:,1]
pred=mod.predict(xts)
prob_lr=pd.DataFrame({'Prob_lr':prob,'Pred_lr':pred})
prob_lgb=pd.read_csv('result_LGB.csv')
results=pd.concat([prob_lr,prob_lgb],axis=1)
results.head()


,Prob_lr,Pred_lr,Unnamed: 0,Prob_LGB,Pred_LGB
0,0.011192,0,0,0.002141,0
1,0.033951,0,1,0.007025,0
2,0.017031,0,2,0.017982,0
3,0.258289,0,3,0.104717,0
4,0.021626,0,4,0.018057,0


In [85]:
results['soft_1:1']=(results['Prob_lr']+results['Prob_LGB'])/2
results['soft_3:2']=(2*results['Prob_lr']+3.5*results['Prob_LGB'])/5.5
results['1:1_pred']=results['soft_1:1'].apply(lambda x:0 if x<0.5 else 1)
results['3:2_pred']=results['soft_3:2'].apply(lambda x:0 if x<0.5 else 1)

results.head()

,Prob_lr,Pred_lr,Unnamed: 0,Prob_LGB,Pred_LGB,soft_1:1,soft_3:2,1:1_pred,3:2_pred
0,0.011192,0,0,0.002141,0,0.006666,0.005432,0,0
1,0.033951,0,1,0.007025,0,0.020488,0.016817,0,0
2,0.017031,0,2,0.017982,0,0.017506,0.017636,0,0
3,0.258289,0,3,0.104717,0,0.181503,0.160561,0,0
4,0.021626,0,4,0.018057,0,0.019841,0.019355,0,0


In [86]:
auc1=roc_auc_score(yts,results['soft_1:1'])
auc2=roc_auc_score(yts,results['soft_3:2'])
recall_1=recall_score(yts,results['1:1_pred'])
recall_2=recall_score(yts,results['3:2_pred'])
print(auc1)
print(auc2)
print(recall_1,recall_2)
print(confusion_matrix(yts,results['1:1_pred']))
print('\n',confusion_matrix(yts,results['3:2_pred']))


0.8861842926471311
0.8831796363471308
0.17162872154115585 0.18739054290718038
[[9221   39]
 [ 473   98]]

 [[9224   36]
 [ 464  107]]


In [70]:
yts.value_counts()

0    9260
1     571
Name: target, dtype: int64

In [89]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=200)
t1=t.drop(rem,axis=1)
v1=v.drop(rem,axis=1)
xtr=t1.drop('target',axis=1)
xts=v1.drop(target,axis=1)
ytr=t1[target]
yts=v1[target]
#cols=list(xtr.columns)
mod=rf.fit(xtr,ytr)
aucroc1(mod,xts,yts)

AUC_ROC=  0.7954393981231063
Accuracy=  0.9412063879564643
confusion matrix:
 [[9237   23]
 [ 555   16]]
